# FIFA23 Card Rating Prediction

In [88]:
# Importing necessary libraries for data manipulation and visualization
import pandas as pd
import numpy as np
import datetime

# Importing libraries for data visualization
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Importing libraries for data preprocessing and modeling
import scipy
from sklearn import preprocessing
from sklearn.datasets import load_digits
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Importing libraries for geographical data visualization
import geopandas as gpd
import folium
from folium.plugins import FastMarkerCluster

# Importing library for one-hot encoding
from sklearn.preprocessing import OneHotEncoder

# Importing library for decision tree and ensemble models
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Importing library for IPython configurations
from IPython import get_ipython

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [89]:
# Load the datasets
fifa_data = pd.read_csv('fifa_2023.csv')

## Exploratory Data Analysis (EDA)

### Explore the dataset

In [90]:
# Display the first few rows of the dataset
print("FIFA23 Dataset:")
fifa_data.head()

FIFA23 Dataset:


,Unnamed: 0,name,rating,card_type,position,nation,league,team,PAC,SHO,PAS,DRI,DEF,PHY
0,0,Lionel Messi,98,team-of-the-season,ST,Argentina,Ligue 1 Uber Eats,Paris SG,94,97,96,99,40,79
1,1,Lionel Messi,98,team-of-the-year,RW,Argentina,Ligue 1 Uber Eats,Paris SG,93,98,97,99,40,77
2,2,Pelé,98,icon,CAM,Brazil,Icons,Icons,95,96,93,96,60,76
3,3,Jude Bellingham,97,award-winner,CM,England,Bundesliga,Borussia Dortmund,90,90,94,98,93,94
4,4,Victor Osimhen,97,award-winner,ST,Nigeria,Serie A TIM,Napoli,99,97,82,94,55,95


In [91]:
# show the shape of the dataset
print("Shape of the dataset:") 
print(fifa_data.shape)

Shape of the dataset:
(10000, 14)


In [92]:
# Check the data types of each column
print("Data types of each column:")
print(fifa_data.dtypes)

Data types of each column:
Unnamed: 0     int64
name          object
rating         int64
card_type     object
position      object
nation        object
league        object
team          object
PAC            int64
SHO            int64
PAS            int64
DRI            int64
DEF            int64
PHY            int64
dtype: object


In [93]:
# Check for missing values
print("Missing values:")
print(fifa_data.isnull().sum())

Missing values:
Unnamed: 0      0
name            0
rating          0
card_type       0
position        0
nation          0
league        379
team          350
PAC             0
SHO             0
PAS             0
DRI             0
DEF             0
PHY             0
dtype: int64


In [94]:
# Show duplicate rows
print("Duplicate rows:")
print(fifa_data[fifa_data.duplicated()])

Duplicate rows:
Empty DataFrame
Columns: [Unnamed: 0, name, rating, card_type, position, nation, league, team, PAC, SHO, PAS, DRI, DEF, PHY]
Index: []


In [95]:
# Showing data with missing values
print("Data with missing values:")
fifa_data[fifa_data.isnull().any(axis=1)]

Data with missing values:


,Unnamed: 0,name,rating,card_type,position,nation,league,team,PAC,SHO,PAS,DRI,DEF,PHY
1230,1230,Dimitri Payet,87,rulebreakers,CAM,France,NaN,OM,87,86,81,86,44,78
1401,1401,Amine Harit,86,world-cup-showdown,CAM,Morocco,NaN,OM,86,80,85,88,50,59
1405,1405,Mattéo Guendouzi,86,world-cup-phenoms,CM,France,NaN,OM,81,80,85,83,84,86
1765,1765,Filip Kostić,85,rare,LM,Serbia,NaN,NaN,84,78,82,82,68,80
1954,1954,Chancel Mbemba,84,ha-fuf8e,CB,DR Congo,NaN,OM,78,52,61,69,84,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9949,9949,Gao Zhunyi,67,common,LB,China PR,NaN,NaN,70,49,60,61,64,72
9951,9951,Marcel Correia,67,common,CB,Portugal,NaN,NaN,59,38,61,62,66,64
9953,9953,Emanuel Aiwu,67,common,CB,Austria,NaN,NaN,70,42,54,56,65,75
9968,9968,Filippo De Col,67,common,RB,Italy,NaN,NaN,76,30,51,57,63,74


### Data Visualization

#### 1. **Rating Distribution**

In [96]:
# Create a histogram to visualize the distribution of player ratings
fig_hist = px.histogram(fifa_data, x="rating", nbins=20)
fig_hist.update_layout(
    title_text='Distribution of Player Ratings',
    xaxis_title_text='Player Rating',
    yaxis_title_text='Count',
    bargap=0.02, # gap between bars of adjacent location coordinates
    bargroupgap=0.01 # gap between bars of the same location coordinates
)
fig_hist.show()

# Create a boxplot to visualize the distribution of player ratings
fig_box = px.box(fifa_data, x="rating")
fig_box.update_layout(
    title_text='Distribution of Player Ratings',
    xaxis_title_text='Player Rating',
    yaxis_title_text='Count',
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig_box.show()

#### 2. **Card Type Distribution**

In [97]:
# Count the number of occurrences of each card type
card_type_counts = fifa_data['card_type'].value_counts()

# Select the top 10 card types
top_10_card_types = card_type_counts.head(10)

# Create a bar chart using Plotly Express
fig_top10card = px.bar(top_10_card_types, 
                       x=top_10_card_types.index, 
                       y=top_10_card_types.values, 
                       color=top_10_card_types.index, 
                       color_discrete_sequence=px.colors.qualitative.Pastel)

fig_top10card.update_layout(title='Distribution of Top 10 Card Types', xaxis_title='Card Type', yaxis_title='Count')
fig_top10card.show()

# Select the top 10 card types and group the rest as "Other"
top_10_card_types = card_type_counts.head(5)
other_count = card_type_counts[5:].sum()
top_10_card_types['Other'] = other_count

# Create a pie chart using Plotly Graph Objects
fig_top10card_pie = go.Figure(data=[go.Pie(labels=top_10_card_types.index, 
                                           values=top_10_card_types.values,
                                           textinfo='label+percent', hole=0.3, 
                                           marker_colors=px.colors.qualitative.Pastel)])
fig_top10card_pie.update_layout(title='Distribution of Top 10 Card Types and Other')
fig_top10card_pie.show()

#### 3. **Position Analysis**

In [98]:
# print the unique values of the position column
print("Unique values of the position column:")
print(fifa_data['position'].unique())

Unique values of the position column:
['ST' 'RW' 'CAM' 'CM' 'CDM' 'LW' 'CF' 'CB' 'LB' 'RB' 'RM' 'LWB' 'RWB' 'LM']


In [99]:
# Count the number of occurrences of each position
position_counts = fifa_data['position'].value_counts()

# Create a bar chart using Plotly Express
fig_position_bar = px.bar(position_counts, 
                          x=position_counts.index, 
                          y=position_counts.values, 
                          color=position_counts.index, 
                          color_discrete_sequence=px.colors.qualitative.Pastel)

fig_position_bar.update_layout(title='Distribution of Players by Position', xaxis_title='Position', yaxis_title='Count')
fig_position_bar.show()

# Create a pie chart using Plotly Graph Objects
fig_position_pie = go.Figure(data=[go.Pie(labels=position_counts.index, 
                                          values=position_counts.values,
                                          textinfo='label+percent', hole=0.3, 
                                          marker_colors=px.colors.qualitative.Pastel)])
fig_position_pie.update_layout(title='Distribution of Players by Position')
fig_position_pie.show()

In [100]:
# Define the roles based on the positions
attacker_positions = ['ST', 'RW', 'LW', 'CF']
midfielder_positions = ['CAM', 'CM', 'CDM', 'RM', 'LM']
defender_positions = ['CB', 'LB', 'RB', 'LWB', 'RWB']

# Classify the positions into roles
fifa_data['role'] = fifa_data['position'].apply(lambda x: 'Attacker' if x in attacker_positions else ('Midfielder' if x in midfielder_positions else 'Defender'))

# Count the number of occurrences of each role
role_counts = fifa_data['role'].value_counts()

# Create a bar chart using Plotly Express
fig_role_bar = px.bar(role_counts, 
                      x=role_counts.index, 
                      y=role_counts.values, 
                      color=role_counts.index, 
                      color_discrete_sequence=px.colors.qualitative.Pastel)

fig_role_bar.update_layout(title='Distribution of Players by Role', xaxis_title='Role', yaxis_title='Count')
fig_role_bar.show()

# Create a pie chart using Plotly Graph Objects
fig_role_pie = go.Figure(data=[go.Pie(labels=role_counts.index, 
                                      values=role_counts.values,
                                      textinfo='label+percent', hole=0.3, 
                                      marker_colors=px.colors.qualitative.Pastel)])
fig_role_pie.update_layout(title='Distribution of Players by Role')
fig_role_pie.show()

#### 4. **Rating Analysis by Nation**

In [101]:
import folium
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Count the number of occurrences of each nation
nation_counts = fifa_data['nation'].value_counts()

# Create a geolocator object
geolocator = Nominatim(user_agent='myapplication')

# Create a world map using Folium
world_map = folium.Map(location=[0, 0], zoom_start=2)

for i in range(len(nation_counts)):
    nation = nation_counts.index[i]
    count = nation_counts.values[i]
    tooltip = f"{nation}: {count} players"
    try:
        location = geolocator.geocode(nation, timeout=10)
        if location:
            lat = location.latitude
            lon = location.longitude
            folium.Marker(location=[lat, lon], tooltip=tooltip).add_to(world_map)
    except GeocoderTimedOut as e:
        print("Error: geocode failed on input %s with message %s"%(nation, e))

folium.map.Marker(location=[0, 0], icon=folium.Icon(color='red'), tooltip='Distribution of Players by Nation').add_to(world_map)

title_html = '''
             <h3 align="center" style="font-size:20px"><b>Distribution of Players by Nation</b></h3>
             '''
world_map.get_root().html.add_child(folium.Element(title_html))
world_map.save('world_map.html')

In [102]:
# Create a new dataframe with the number of players from each nation
nation_counts = fifa_data['nation'].value_counts()

# Create a world map using Plotly Express with a darker blue-green color scale
fig_nation_map = px.choropleth(locations=nation_counts.index, 
                               locationmode='country names', 
                               color=nation_counts.values, 
                               title='Distribution of Players by Nation',
                               color_continuous_scale='blues',
                               color_continuous_midpoint=100)

# Set the height and width of the figure
fig_nation_map.update_layout(height=800, width=1200)

fig_nation_map.show()

# Create a bar chart using Plotly Express
fig_nation_bar = px.bar(nation_counts.head(20), 
                        x=nation_counts.head(20).index, 
                        y=nation_counts.head(20).values, 
                        color=nation_counts.head(20).index, 
                        color_discrete_sequence=px.colors.qualitative.Pastel)

fig_nation_bar.update_layout(title='Top 20 Nations with the Most Players', xaxis_title='Nation', yaxis_title='Count')

# Show the bar chart
fig_nation_bar.show()

In [103]:
import pycountry_convert as pc

def classify_nation(nation):
    try:
        country_alpha2 = pc.country_name_to_country_alpha2(nation)
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except:
        return 'Unknown'

# Classify the nations into continents
fifa_data['continent'] = fifa_data['nation'].apply(lambda x: classify_nation(x))

# Count the number of occurrences of each continent
continent_counts = fifa_data['continent'].value_counts()

# Create a bar chart using Plotly Express
fig_continent_bar = px.bar(continent_counts,
                            x=continent_counts.index,
                            y=continent_counts.values,
                            color=continent_counts.index,
                            color_discrete_sequence=px.colors.qualitative.Pastel)

fig_continent_bar.update_layout(title='Distribution of Players by Continent', xaxis_title='Continent', yaxis_title='Count')

# Show the bar chart
fig_continent_bar.show()

# Create a pie chart using Plotly Graph Objects
fig_continent_pie = go.Figure(data=[go.Pie(labels=continent_counts.index,
                                            values=continent_counts.values,
                                            textinfo='label+percent', hole=0.3,
                                            marker_colors=px.colors.qualitative.Pastel)])
                                                
fig_continent_pie.update_layout(title='Distribution of Players by Continent')
fig_continent_pie.show()

In [104]:
# print nation for every Unknown continent
print("Nations for Unknown continent:")
print(fifa_data[fifa_data['continent'] == 'Unknown']['nation'].unique())


Nations for Unknown continent:
['England' 'Korea Republic' 'Scotland' 'Northern Ireland'
 'Republic of Ireland' 'Wales' 'Bosnia Herzegovina' 'DR Congo' 'Kosovo'
 'FYR Macedonia' 'Trinidad & Tobago' 'CAR' 'Luxemburg'
 'Cape Verde Islands' 'St Kitts Nevis' 'China PR' 'Korea DPR'
 'Netherlands Antilles' 'Chinese Taipei' 'Antigua & Barbuda']


In [105]:
# Since the continent of the nations are unknown, we will manually classify them
def update_continents(df, nation_to_continent):
    for nation, continent in nation_to_continent.items():
        df.loc[df['nation'] == nation, 'continent'] = continent

update_continents(fifa_data, {
    'England': 'Europe',
    'Wales': 'Europe',
    'Scotland': 'Europe',
    'Northern Ireland': 'Europe',
    'Korea Republic': 'Asia',
    'Ivory Coast': 'Africa',
    'Republic of Ireland': 'Europe',
    'Bosnia Herzegovina': 'Europe',
    'DR Congo': 'Africa',
    'Cape Verde': 'Africa',
    'FYR Macedonia': 'Europe',
    'St Kitts Nevis': 'North America',
    'St Lucia': 'North America',
    'St Vincent Grenadines': 'North America',
    'Guinea Bissau': 'Africa',
    'Trinidad & Tobago': 'North America',
    'Antigua & Barbuda': 'North America',
    'Central African Rep.': 'Africa',
    'Congo': 'Africa',
    'Korea DPR': 'Asia',
    'São Tomé & Príncipe': 'Africa',
    'Gambia': 'Africa',
    'Chinese Taipei': 'Asia',
    'Curaçao': 'North America'
})


In [106]:
# Count the number of occurrences of each continent
continent_counts = fifa_data['continent'].value_counts()

# Create a bar chart using Plotly Express
fig_continent_bar = px.bar(continent_counts,
                            x=continent_counts.index,
                            y=continent_counts.values,
                            color=continent_counts.index,
                            color_discrete_sequence=px.colors.qualitative.Pastel)

fig_continent_bar.update_layout(title='Distribution of Players by Continent', xaxis_title='Continent', yaxis_title='Count')

# Show the bar chart
fig_continent_bar.show()

# Create a pie chart using Plotly Graph Objects
fig_continent_pie = go.Figure(data=[go.Pie(labels=continent_counts.index,
                                            values=continent_counts.values,
                                            textinfo='label+percent', hole=0.3,
                                            marker_colors=px.colors.qualitative.Pastel)])
                                                
fig_continent_pie.update_layout(title='Distribution of Players by Continent')
fig_continent_pie.show()

In [107]:
import folium
from geopy.geocoders import Nominatim

# Count the number of occurrences of each nation
nation_counts = fifa_data['nation'].value_counts()

# Define a dictionary that maps nation names to their latitude and longitude coordinates
nation_coordinates = {
    'England': [52.3555, -1.1743],
    'Korea Republic': [35.9078, 127.7669],
    'Scotland': [56.4907, -4.2026],
    'Northern Ireland': [54.7877, -6.4923],
    'Republic of Ireland': [53.1424, -7.6921],
    'Wales': [52.1307, -3.7837],
    'Bosnia Herzegovina': [43.9159, 17.6791],
    'DR Congo': [-4.0383, 21.7587],
    'Kosovo': [42.6026, 20.9029],
    'FYR Macedonia': [41.6086, 21.7453],
    'Trinidad & Tobago': [10.6918, -61.2225],
    'CAR': [6.6111, 20.9394],
    'Luxemburg': [49.8153, 6.1296],
    'Cape Verde Islands': [15.1201, -23.6052],
    'St Kitts Nevis': [17.3578, -62.7822],
    'China PR': [35.8617, 104.1954],
    'Korea DPR': [40.3399, 127.5101],
    'Netherlands Antilles': [12.2019, -68.2624],
    'Chinese Taipei': [23.6978, 120.9605],
    'Antigua & Barbuda': [17.0608, -61.7964]
}

# Create a geolocator object
geolocator = Nominatim(user_agent='myapplication')

# Create a world map using Folium
world_map = folium.Map(location=[0, 0], zoom_start=2)

# Add markers for each nation
for i in range(len(nation_counts)):
    nation = nation_counts.index[i]
    count = nation_counts.values[i]
    tooltip = f"{nation}: {count} players"
    if nation in nation_coordinates:
        lat = nation_coordinates[nation][0]
        lon = nation_coordinates[nation][1]
        folium.Marker(location=[lat, lon], tooltip=tooltip).add_to(world_map)
    else:
        location = geolocator.geocode(nation)
        if location:
            lat = location.latitude
            lon = location.longitude
            folium.Marker(location=[lat, lon], tooltip=tooltip).add_to(world_map)

folium.map.Marker(location=[0, 0], icon=folium.Icon(color='red'), tooltip='Distribution of Players by Nation').add_to(world_map)

title_html = '''
             <h3 align="center" style="font-size:20px"><b>Distribution of Players by Nation</b></h3>
             '''
world_map.get_root().html.add_child(folium.Element(title_html))
world_map.save('world_map.html')

#### 5. **League Analysis**

In [108]:
# Count the number of occurrences of each league
league_counts = fifa_data['league'].value_counts().head(10)

# Create a bar chart using Plotly Express
fig_league_bar = px.bar(league_counts, 
                        x=league_counts.index, 
                        y=league_counts.values, 
                        color=league_counts.index, 
                        color_discrete_sequence=px.colors.qualitative.Pastel)

fig_league_bar.update_layout(title='Top 10 Leagues by Player', xaxis_title='League', yaxis_title='Count')

# Create a pie chart using go.Pie()
fig_league_pie = go.Figure(data=[go.Pie(labels=league_counts.index, 
                                         values=league_counts.values, 
                                         hole=0.3, 
                                         textinfo='label+percent',
                                         marker=dict(colors=px.colors.qualitative.Pastel))])

fig_league_pie.update_layout(title='Top 10 Leagues by Player')

# Show the charts
fig_league_bar.show()
fig_league_pie.show()

#### 6. **Player Statistic Analysis**

In [109]:
# create a boxplot to visualize the distribution of player ratings PAC, SHO, PAS, DRI, DEF, and PHY
stat_columns = ['PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY']

# set a different color for each box
color_sequence = px.colors.qualitative.Pastel[:len(stat_columns)]

fig_box = go.Figure()
for i, stat in enumerate(stat_columns):
    fig_box.add_trace(go.Box(y=fifa_data[stat], name=stat, boxmean=True, marker_color=color_sequence[i]))

fig_box.update_layout(
    title_text='Distribution of Player Ratings',
    xaxis_title_text='Player Ratings',
    yaxis_title_text='Count',
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig_box.show()

In [110]:
import plotly.graph_objects as go

# create a violin plot to visualize the distribution of player ratings PAC, SHO, PAS, DRI, DEF, and PHY
stat_columns = ['PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY']

# set a different color for each violin
color_sequence = px.colors.qualitative.Pastel[:len(stat_columns)]

fig_violin = go.Figure()
for i, stat in enumerate(stat_columns):
    fig_violin.add_trace(go.Violin(y=fifa_data[stat], name=stat, marker_color=color_sequence[i]))

fig_violin.update_layout(
    title_text='Distribution of Player Ratings',
    xaxis_title_text='Player Ratings',
    yaxis_title_text='Count',
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig_violin.show()

In [111]:
# sort the most name shown in name column
name_counts = fifa_data['name'].value_counts().sort_values(ascending=False)

# create a bar chart to visualize the distribution of player names
fig_name_bar = px.bar(name_counts.head(15),
                        x=name_counts.head(15).index,
                        y=name_counts.head(15).values,
                        color=name_counts.head(15).index,
                        color_discrete_sequence=px.colors.qualitative.Pastel)

fig_name_bar.update_layout(title='Top 15 Players by Name', xaxis_title='Name', yaxis_title='Count')

fig_name_bar.show()

In [112]:
def hexagraph(name, card_type):
    # Filter the data to get the player's attributes
    player_data = fifa_data[(fifa_data['name'] == name) & (fifa_data['card_type'].str.contains(card_type))]
    attributes = ['PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY']
    values = [player_data[attr].values[0] for attr in attributes]

    # Create a hexagraph chart using Plotly Express
    fig = px.line_polar(theta=attributes + [attributes[0]], r=values + [values[0]], line_close=True)

    # Set the title and axis labels
    card_type_formatted = card_type.replace('-', ' ').title()
    fig.update_layout(title=f'{name} ({card_type_formatted}) Attribute Distribution', polar=dict(radialaxis=dict(visible=True, showline=False, showticklabels=False)))

    # Show the figure
    fig.show()

In [113]:
best_players = fifa_data.loc[fifa_data.groupby('role')['rating'].idxmax(), ['role', 'name', 'rating', 'card_type']]
best_players

,role,name,rating,card_type
0,Attacker,Lionel Messi,98,team-of-the-season
29,Defender,Matthijs de Ligt,96,team-of-the-season
2,Midfielder,Pelé,98,icon


In [114]:
# Loop over each row in the best_players DataFrame and display a hexagraph chart for each player
for i in range(len(best_players)):
    # Get the name and card_type of the player at the current row
    name = best_players.iloc[i]['name']
    card_type = best_players.iloc[i]['card_type']
    
    # Display a hexagraph chart for the current player
    hexagraph(name, card_type)

#### 7. **Team Analysis**

In [115]:
# Sort the data by the number of players in descending order
team_counts = fifa_data['team'].value_counts().sort_values(ascending=False)

# Create a new bar chart showing the distribution of teams by the number of players
fig_team_bar = px.bar(team_counts.head(10),
                        x=team_counts.head(10).index,
                        y=team_counts.head(10).values,
                        color=team_counts.head(10).index,
                        color_discrete_sequence=px.colors.qualitative.Pastel)

# Set the title and axis labels
fig_team_bar.update_layout(title='Top 10 Teams by Number of Players', xaxis_title='Team', yaxis_title='Number of Players')

# Show the figure
fig_team_bar.show()

In [116]:
# Exclude 'Icons' and 'HERO' teams from the data
team_counts = fifa_data[fifa_data['team'].isin(['Icons', 'HERO']) == False]['team'].value_counts().sort_values(ascending=False)

# Create a new bar chart showing the distribution of teams by the number of players
fig_team_bar = px.bar(team_counts.head(10),
                        x=team_counts.head(10).index,
                        y=team_counts.head(10).values,
                        color=team_counts.head(10).index,
                        color_discrete_sequence=px.colors.qualitative.Pastel)

# Set the title and axis labels
fig_team_bar.update_layout(title='Top 10 Teams by Number of Players (Excluding Icons and HERO)', xaxis_title='Team', yaxis_title='Number of Players')

# Show the figure
fig_team_bar.show()

## Data Preprocessing

## Model Building

## Model Training

## Prediction Test